##### Copyright 2022 The Cirq Developers

In [3]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello Qubit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/start/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [4]:
import sys
sys.meta_path[:] = [f for f in sys.meta_path if "DaskFinder" not in str(f)]

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

import numpy as np
import matplotlib.pyplot as plt

In [5]:
#arbirary angles
theta_1 = 0
theta_2 = 0
theta_3 = 0

#Form curcuit
qubits = [cirq.LineQubit(i) for i in range(4)]
circuit = cirq.Circuit()

#Append 20 gates
circuit.append(cirq.Z(qubits[0]))
circuit.append(cirq.Y(qubits[1]))
circuit.append(cirq.H(qubits[2]))
circuit.append(cirq.T(qubits[3]))
circuit.append(cirq.X(qubits[0]))
circuit.append(cirq.X(qubits[1]))
circuit.append(cirq.S(qubits[3]))
circuit.append(cirq.X(qubits[0]))
circuit.append(cirq.CNOT(qubits[3], qubits[2]))
circuit.append(cirq.CZ(qubits[0], qubits[1]))
circuit.append(cirq.S(qubits[2]))
circuit.append(cirq.X(qubits[3]))
circuit.append(cirq.SWAP(qubits[2], qubits[3]))
circuit.append(cirq.CNOT(qubits[2], qubits[3]))
circuit.append(cirq.CCNOT(qubits[2], qubits[1], qubits[0]))
circuit.append(cirq.CZ(qubits[2], qubits[3]))
circuit.append(cirq.#(qubits[]))
circuit.append(cirq.#(qubits[]))
circuit.append(cirq.#(qubits[]))
circuit.append(cirq.#(qubits[]))


# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=10100101000000010110


# Congratulations
You've just run your first Cirq program.

To learn about running a circuit on a virtual machine that mimics existing quantum hardware, see [Quantum Virtual Machine](../simulate/quantum_virtual_machine.ipynb).

If you would like to learn more about quantum computing, check out our [education page](https://quantumai.google/resources). The Full API reference for Cirq can be found [here](/reference/python/cirq). If you are looking for vendor specific information that can be found on our vendor sub-pages:


  [Alpine Quantum Technologies](../hardware/aqt/getting_started.ipynb)
  
  [Pasqal](../hardware/pasqal/getting_started.ipynb)
  
  [IonQ](../hardware/ionq/getting_started.ipynb)
  
  [Azure](../hardware/azure-quantum/getting_started_honeywell.ipynb)
  
  [Rigetti](../hardware/rigetti/getting_started.ipynb)